In [1]:
!pip install kaggle

In [2]:
import io, os

import googleapiclient.discovery
import googleapiclient.http
import google.colab

def colab_kaggle_install_apikey():
  """ 
  Copies 'kaggle.json' from Google Drive to Colaboratory environment
  
  Based on https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
  """
  
  google.colab.auth.authenticate_user()

  drive_service = googleapiclient.discovery.build('drive', 'v3')
  results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
  kaggle_api_key = results.get('files', [])
  if len(kaggle_api_key) == 0:
    raise ValueError("Could not find kaggle.json in Google Drive")

  filename = "/root/.kaggle/kaggle.json"
  os.makedirs(os.path.dirname(filename), exist_ok=True)

  request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
  fh = io.FileIO(filename, 'wb')
  downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
      status, done = downloader.next_chunk()
  os.chmod(filename, 600)
  print('Installed to {}'.format(filename))
  
colab_kaggle_install_apikey()

Installed to /root/.kaggle/kaggle.json


In [3]:
!kaggle competitions download -c dat300-2018-concrete -p data/

Concrete_sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
Concrete_test.csv: Skipping, found more recently modified local copy (use --force to force download)
Concrete_train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import time

import pandas
import numpy

import sklearn

import sklearn.preprocessing
import sklearn.model_selection
import sklearn.pipeline

import sklearn.linear_model
import sklearn.ensemble
import sklearn.svm

import keras
import keras.wrappers.scikit_learn

In [5]:
dataset = pandas.read_csv('data/Concrete_train.csv')
target_column = 'ConcreteComp
ressiveStrength'
feature_columns = list(set(dataset.columns) - set([target_column]))

dataset.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,ConcreteCompressiveStrength
0,525.000000,0.000000,0.000000,189.000000,0.0,1125.000000,613.000000,7,42.419998
1,276.000000,116.000000,90.000000,180.000000,9.0,870.000000,768.000000,28,44.279999
2,182.000000,45.200001,122.000000,170.199997,8.2,1059.400024,780.700012,100,48.669998
3,212.600006,0.000000,100.400002,159.399994,10.4,1003.799988,903.799988,100,47.740002
4,251.399994,0.000000,118.300003,188.500000,6.4,1028.400024,757.700012,100,44.209999


In [6]:
compete = pandas.read_csv('data/Concrete_test.csv')
compete.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age
0,540.000000,0.000000,0.0,173.0,0.0,1125.000000,613.000000,28
1,314.000000,145.000000,113.0,179.0,8.0,869.000000,690.000000,28
2,302.000000,0.000000,0.0,203.0,0.0,974.000000,817.000000,14
3,166.800003,250.199997,0.0,203.5,0.0,975.599976,692.599976,3
4,446.000000,24.000000,79.0,162.0,10.3,967.000000,712.000000,56


https://www.kaggle.com/c/dat300-2018-concrete

Keras to train a multilayer perceptron predicting the strength of the concrete.
    
        at least two types of activation functions
        and use two different network complexities
        (number of layers and number of hidden units in each of them).
        
        Compare and discuss (in a few senteces) ANN results
        (performance and time consumption)
        to results from linear regression method in scikit-learn
        (with polynomials, for example linear, quadratic, cubic, etc.).

        You can submit predictions to Kaggle from any of these models.


* MLP: Two different architectures, two different activation functions
* sklearn regression. PolynomialFeatures+Ridge, SVR rbf/poly

In [0]:
#test.shape, train.shape

In [83]:
# like sklearn.model_evaluation.cross_val_scores, but not requiring cloning of estimator
def cv_scores(estimator, X, Y_true, cv=5, metric=None):
  
    if metric is None:
      metric = sklearn.metrics.mean_absolute_error
    
    kf = sklearn.model_selection.KFold(n_splits=cv)

    scores = []
    for train_index, test_index in kf.split(X):
      X_test = X[test_index]
      Y_true_test = Y_true[test_index]
      Y_pred = estimator.predict(X_test)
      scores.append(metric(Y_true_test, Y_pred))
      
    return scores
    

def train_evaluate_model(dataset, model, gridparams, gridcv=5, evalcv=10, test_size=0.3, rng=1):
  
    scoring = 'neg_mean_absolute_error'
  
    X = dataset[feature_columns]
    X = sklearn.preprocessing.RobustScaler().fit_transform(X)
    
    Y = dataset[target_column].values
    X_train, X_test, Y_train, Y_test = \
      sklearn.model_selection.train_test_split(X, Y, test_size=test_size, random_state=rng)
    
 
    # Find hyperparameters
    train_start = time.time()
    if gridparams is not None:
      grid = sklearn.model_selection.GridSearchCV(model, gridparams, cv=gridcv, scoring=scoring,
                                                  iid=False, refit=True, return_train_score=True)
      grid.fit(X_train, Y_train)
      estimator = grid.best_estimator_
      details = grid.cv_results_
    else:
      model.fit(X_train, Y_train)
      estimator = model
      details = {}
    
    train_time = time.time() - train_start
    
    # Evaluate   
    evaluate_start = time.time()
    test_scores = cv_scores(estimator, X_test, Y_test, cv=evalcv)
    train_scores = cv_scores(estimator, X_train, Y_train, cv=evalcv)
    evaluate_time =  time.time() - evaluate_start
   
    
    return test_scores, train_scores, details, train_time, evaluate_time

def build_mlp(layer_sizes, activation='relu'):
    Dense = keras.layers.Dense
  
    model = keras.Sequential()
    for i, size in enumerate(layer_sizes):
      params = dict(activation=activation)
      if i == 0:
        params['input_dim'] = 8
        model.add(Dense(layer_sizes[0], **params))  
    model.add(Dense(1))
    
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model

def KerasMLP(layer_sizes, activation):
  def build_func():
    return build_mlp(layer_sizes, activation)
  return keras.wrappers.scikit_learn.KerasRegressor(build_func, epochs=50, batch_size=5, verbose=1)
  
def PolyRidge(degree):
    m = sklearn.pipeline.make_pipeline(
      sklearn.preprocessing.PolynomialFeatures(degree),
      sklearn.linear_model.Ridge(),
    )
    return m
 
C_params = numpy.geomspace(0.001, 1000.0, 50)
A_params = numpy.geomspace(0.001, 1000.0, 50)

models = {
    'MLP 8-8 ReLu': ( KerasMLP([8,8],'relu'), None ),
    'MLP 8-8 tanh': ( KerasMLP([8,8],'tanh'), None ),
    'MLP 16-16 Relu': ( KerasMLP([16,16],'relu'), None ),
    'Poly3-Ridge': ( PolyRidge(3) , { 'ridge__alpha': A_params }),
    'Poly2-Ridge': ( PolyRidge(2) , { 'ridge__alpha': A_params }),
    'RandomForest': ( sklearn.ensemble.RandomForestRegressor(n_estimators=50), {'min_samples_leaf': [1, 0.01, 0.03, 0.05, 0.10]}),
    'SVM poly3': ( sklearn.svm.SVR(kernel='poly', degree=3), { 'C': C_params } ),
    'SVM poly2': ( sklearn.svm.SVR(kernel='poly', degree=2), { 'C': C_params } ),
    'SVM rbf': ( sklearn.svm.SVR(kernel='rbf', degree=2), { 'C': C_params } ),
}

results = []
for i, model in enumerate(models.items()):
    name, values = model
    estimator, params = values
    r = train_evaluate_model(dataset, estimator, params)
    assert len(r) == 5, len(r)
    print('r', i, r[3], r[4])
    results.append((name, r))

Epoch 1/50
504/504 [==============================] - 5s 10ms/step - loss: 34.7997
Epoch 2/50
504/504 [==============================] - 0s 594us/step - loss: 34.0949
Epoch 3/50
504/504 [==============================] - 0s 600us/step - loss: 33.1242
Epoch 4/50
504/504 [==============================] - 0s 602us/step - loss: 31.7748
Epoch 5/50
504/504 [==============================] - 0s 602us/step - loss: 29.9881
Epoch 6/50
504/504 [==============================] - 0s 574us/step - loss: 27.7334
Epoch 7/50
504/504 [==============================] - 0s 566us/step - loss: 24.9422
Epoch 8/50
504/504 [==============================] - 0s 567us/step - loss: 21.8886
Epoch 9/50
504/504 [==============================] - 0s 575us/step - loss: 18.6908
Epoch 10/50
504/504 [==============================] - 0s 574us/step - loss: 15.7407
Epoch 11/50
504/504 [==============================] - 0s 587us/step - loss: 13.0532
Epoch 12/50
504/504 [==============================] - 0s 595us/step - loss

In [0]:
:


In [84]:
test_scores = numpy.array([r[1][0] for r in results])
train_scores = numpy.array([r[1][1] for r in results])

def selected_params(r):
    details = r[1][2]
    if 'mean_test_score' in details.keys():
      df = pandas.DataFrame(details)
      s = df.sort_values('rank_test_score', ascending=True)
      return s.params.iloc[0]
    else:
      return None

df = pandas.DataFrame({
    'test_mean': test_scores.mean(axis=1),
    'test_std': test_scores.std(axis=1),
    'train_mean': train_scores.mean(axis=1),
    'train_std': train_scores.std(axis=1),
    'model': [ r[0] for r in results ],
    'train_time': [ r[1][3] for r in results ],
    'predict_time': [ r[1][4] for r in results ],
    'params': [ selected_params(r) for r in results ],
})
df.sort_values(by='test_mean', ascending=True).head(10)

,model,params,predict_time,test_mean,test_std,train_mean,train_std,train_time
5,RandomForest,{'min_samples_leaf': 1},0.050932,4.137398,0.474259,1.595700,0.301961,2.311819
8,SVM rbf,{'C': 104.81131341546852},0.015871,4.510368,0.960680,2.789694,0.633476,11.609962
3,Poly3-Ridge,{'ridge__alpha': 1.151395399326447},0.044537,5.460879,1.436678,3.288003,0.450783,4.896723
2,MLP 16-16 Relu,None,2.542748,6.097163,1.564195,5.447361,0.850862,20.157034
6,SVM poly3,{'C': 1000.0},0.008908,6.167713,1.777015,4.122659,0.830140,20.127264
4,Poly2-Ridge,{'ridge__alpha': 0.28117686979742307},0.017375,6.655942,1.185258,5.234722,0.944606,1.874300
7,SVM poly2,{'C': 104.81131341546852},0.008672,7.370464,1.420235,5.916535,0.888379,9.163062
1,MLP 8-8 tanh,None,2.500424,7.808446,1.039799,8.048932,0.853400,19.930687
0,MLP 8-8 ReLu,None,2.462641,7.882853,2.286932,6.422631,1.226349,20.256497
